In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
# Reading the csv files into corresponding dataframes
# Then reduce their size to consume less memory
startTime = datetime.now()
data_directory_path = 'C:/Users/saidh/Downloads/Market Basket Analysis/'
aisles = pd.read_csv(data_directory_path + 'aisles.csv')

departments = pd.read_csv(data_directory_path + 'departments.csv')

order_products_prior = pd.read_csv(data_directory_path + 'order_products__prior.csv')

order_products_train = pd.read_csv(data_directory_path + 'order_products__train.csv')

orders = pd.read_csv(data_directory_path + 'orders.csv')
# Replacing numbers with their corresponding hour representation 

products = pd.read_csv(data_directory_path + 'products.csv')

In [2]:
order_products = pd.concat([order_products_train, order_products_prior])
order_products = order_products.merge(products, on='product_id', how='left').merge(orders, on='order_id', how='left').merge(departments, on='department_id').merge(aisles, on='aisle_id')

df=order_products
df = df[df['eval_set'] == 'train']
df

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,department,aisle
0,1,49302,1,1,Bulgarian Yogurt,120,16,112108,train,4,4,10,9.0,dairy eggs,yogurt
1,98,43654,3,1,Whole Milk Greek Blended Vanilla Bean Yogurt,120,16,56463,train,41,3,8,14.0,dairy eggs,yogurt
2,170,8153,6,0,Plain Non-Fat Greek Yogurt,120,16,182389,train,7,0,13,14.0,dairy eggs,yogurt
3,631,9203,13,1,Dairy Free Coconut Milk Raspberry Yogurt Alter...,120,16,184099,train,7,3,9,30.0,dairy eggs,yogurt
4,631,15842,14,1,Dairy Free Coconut Milk Yogurt Alternative,120,16,184099,train,7,3,9,30.0,dairy eggs,yogurt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33755680,3413737,3618,2,0,Grilled Seafood Feast Variety Cat Food,41,8,195012,train,55,6,22,0.0,pets,cat food care
33755681,3413737,2727,3,1,Poultry & Beef Feast Variety Cat Food,41,8,195012,train,55,6,22,0.0,pets,cat food care
33755682,3416173,45855,4,0,Prime Filets Seafood Favorites Variety Pack Ca...,41,8,62014,train,5,6,14,15.0,pets,cat food care
33755683,3418238,29010,22,0,Original Cat Litter,41,8,46578,train,6,3,12,15.0,pets,cat food care


In [3]:
product_counts = df.groupby('product_id')['order_id'].count().reset_index().rename(columns = {'order_id':'frequency'})
product_counts = product_counts.sort_values('frequency', ascending=False)[0:100].reset_index(drop=True)
product_counts 


,product_id,frequency
0,24852,18726
1,13176,15480
2,21137,10894
3,21903,9784
4,47626,8135
...,...,...
95,8193,1418
96,9387,1379
97,37687,1362
98,20995,1361


In [4]:
freq_products = list(product_counts.product_id)
del product_counts 
freq_products[1:10]


[13176, 21137, 21903, 47626, 47766, 47209, 16797, 26209, 27966]

In [5]:
order_products = df[df.product_id.isin(freq_products)]
del df
order_products.shape
print(order_products)


          order_id  product_id  add_to_cart_order  reordered  \
1828410          1       22035                  8          1   
1828429       2445       22035                  4          0   
1828430       2445       33731                  7          0   
1828431       2445       40604                 20          1   
1828440       2937       40604                 16          1   
...            ...         ...                ...        ...   
29573237   3415183        5077                 11          1   
29573248   3417341        5077                  3          1   
29573251   3417506        5077                 11          0   
29573278   3420258        5077                  4          1   
29573279   3420286        5077                 26          1   

                         product_name  aisle_id  department_id  user_id  \
1828410   Organic Whole String Cheese        21             16   112108   
1828429   Organic Whole String Cheese        21             16      816   
182843

In [6]:
df=order_products[['order_id','product_name','reordered']].set_index('order_id')
df

,product_name,reordered
order_id,,
1,Organic Whole String Cheese,1
2445,Organic Whole String Cheese,0
2445,Grated Parmesan,0
2445,Feta Cheese Crumbles,1
2937,Feta Cheese Crumbles,1
...,...,...
3415183,100% Whole Wheat Bread,1
3417341,100% Whole Wheat Bread,1
3417506,100% Whole Wheat Bread,0


In [7]:
basket = df.pivot_table(columns='product_name', values='reordered', index='order_id').reset_index().fillna(0).set_index('order_id')
basket

product_name,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Blueberries,Boneless Skinless Chicken Breasts,Broccoli Crown,Bunched Cilantro,...,Sparkling Lemon Water,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420998,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3421049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3421056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket = basket.applymap(encode_units)
basket.head()




product_name,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Blueberries,Boneless Skinless Chicken Breasts,Broccoli Crown,Bunched Cilantro,...,Sparkling Lemon Water,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
shortbasket = basket[:100000]
shortbasket


product_name,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Blueberries,Boneless Skinless Chicken Breasts,Broccoli Crown,Bunched Cilantro,...,Sparkling Lemon Water,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420998,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3421049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3421056,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_items = apriori(shortbasket, min_support=0.001, use_colnames=True  , low_memory=True)
rules = association_rules(frequent_items, metric='lift', min_threshold=1.1)



C:\ProgramData\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [11]:
from pandas import DataFrame
from sqlalchemy import asc


rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))

df=pd.DataFrame(rules)

df.head(10).sort_values(by=['antecedent_len','confidence'],ascending=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(100% Whole Wheat Bread),(Banana),0.018668,0.176420,0.004869,0.260845,1.478546,0.001576,1.114218,1
6,(100% Whole Wheat Bread),(Organic Strawberries),0.018668,0.091668,0.002280,0.122146,1.332490,0.000569,1.034719,1
2,(100% Whole Wheat Bread),(Organic Hass Avocado),0.018668,0.064379,0.001481,0.079338,1.232351,0.000279,1.016248,1
8,(100% Whole Wheat Bread),(Strawberries),0.018668,0.050996,0.001428,0.076484,1.499796,0.000476,1.027599,1
4,(100% Whole Wheat Bread),(Organic Raspberries),0.018668,0.045594,0.001183,0.063356,1.389571,0.000332,1.018964,1
9,(Strawberries),(100% Whole Wheat Bread),0.050996,0.018668,0.001428,0.027998,1.499796,0.000476,1.009599,1
1,(Banana),(100% Whole Wheat Bread),0.176420,0.018668,0.004869,0.027602,1.478546,0.001576,1.009187,1
5,(Organic Raspberries),(100% Whole Wheat Bread),0.045594,0.018668,0.001183,0.025941,1.389571,0.000332,1.007466,1
7,(Organic Strawberries),(100% Whole Wheat Bread),0.091668,0.018668,0.002280,0.024875,1.332490,0.000569,1.006365,1
3,(Organic Hass Avocado),(100% Whole Wheat Bread),0.064379,0.018668,0.001481,0.023006,1.232351,0.000279,1.004440,1


In [12]:
df[(df['confidence']>0.45) & (df['consequents']!={'Bag of Organic Bananas'})].sort_values(by=['confidence'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
3266,"(Strawberries, Organic Fuji Apple)",(Banana),0.001918,0.176420,0.001044,0.544444,3.086073,0.000706,1.807859,2
2964,"(Strawberries, Honeycrisp Apple)",(Banana),0.002664,0.176420,0.001385,0.520000,2.947515,0.000915,1.715792,2
3180,"(Organic Avocado, Strawberries)",(Banana),0.004134,0.176420,0.002003,0.484536,2.746495,0.001274,1.597746,2
3448,"(Sparkling Water Grapefruit, Sparkling Lemon W...",(Lime Sparkling Water),0.002664,0.015727,0.001279,0.480000,30.520325,0.001237,1.892832,2
3450,"(Lime Sparkling Water, Sparkling Lemon Water)",(Sparkling Water Grapefruit),0.002706,0.028364,0.001279,0.472441,16.656117,0.001202,1.841757,2
3139,"(Organic Avocado, Organic Peeled Whole Baby Ca...",(Banana),0.002376,0.176420,0.001076,0.452915,2.567256,0.000657,1.505397,2


In [13]:
rules.dtypes

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
antecedent_len          int64
dtype: object

In [14]:
len(rules)

3868

In [15]:
antecedent_len_count = df.groupby('antecedent_len')['antecedents'].count().reset_index().rename(columns = {'antecedent_len':'antecedent_len'})
antecedent_len_count = antecedent_len_count.sort_values('antecedent_len', ascending=True)
antecedent_len_count

,antecedent_len,antecedents
0,1,2991
1,2,865
2,3,12


In [17]:

from xmlrpc.client import DateTime


rules[(rules['lift'] >= 1.4) & (rules['confidence'] >= 0.3)].sort_values(by=['confidence', 'lift'], ascending=False).to_excel(r'C:/Users/saidh/Downloads/marketbasketanalysisusingapriori.xlsx', index=False)
tt=datetime.now()-startTime
print('It took ',tt, ' to run')

It took  0:02:41.895730  to run
